In [1]:
import kss
import pandas as pd
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification
import torch
from tqdm import tqdm

In [2]:
poem_df = pd.read_csv('/opt/ml/final-project-level3-nlp-08/data/sinchoon+nonpub+400poems.csv')

In [3]:
t = poem_df['text'][0].split('\n')

In [4]:
labels = ['기쁨', '불안', '당황', '슬픔', '분노', '상처']
#labels = valid_df['감정_소분류'].unique().tolist()
label_idx_dict = {label: idx for idx, label in enumerate(labels)}
idx_label_dict = {idx: label for label, idx in label_idx_dict.items()}

In [5]:
model = AutoModelForSequenceClassification.from_pretrained('./results')
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

In [6]:
device = 'cuda'
model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [78]:
#data = kss.split_sentences(poem_df['text'][index])
data = poem_df['text'][index]
data = tokenizer(data,return_tensors='pt',padding=True)
data = {k: v.to(device) for k, v in data.items()}

In [79]:
output = model(**data)

In [10]:
# print(poem_df['text'][index])
# print('-'*50)
# idx_label_dict[torch.argmax(output.logits,1).detach().cpu().item()]
#idx_label_dict[torch.argmax(torch.mean(output.logits,0)).detach().cpu().item()]  
#torch.argmax(output.logits, dim=1).detach().cpu().item()

# torch.softmax(output.logits, dim=1)

In [7]:
p = []
l = []
# len(poem_df)
for index in tqdm(range(len(poem_df))):
    texts = poem_df['text'][index].split('\n')
    data = []
    for text in texts:
        if text != '':
            data.append(text)
    
    data = tokenizer(data,return_tensors='pt',padding=True)
    data = {k: v.to(device) for k, v in data.items()}

    output = model(**data)

    prob = torch.softmax(output.logits, dim=1)
    prob_mean = torch.mean(prob, dim = 0)
    #print(torch.argmax(prob_mean, 0))

    label=idx_label_dict[torch.argmax(prob_mean, 0).detach().cpu().item()]
    p.append(poem_df['text'][index])
    l.append(label)
    #print(poem_df['text'][index], '\n' , label)
    #print()

100%|██████████| 1149/1149 [00:22<00:00, 51.74it/s]


In [9]:
p_t = []
l_t = []
# len(poem_df)
for index in tqdm(range(len(poem_df))):
    texts = poem_df['text'][index]
    data = texts
    
    data = tokenizer(data,return_tensors='pt',padding=True,truncation=True,max_length=512)
    data = {k: v.to(device) for k, v in data.items()}

    output = model(**data)

    prob = torch.softmax(output.logits, dim=1)
    prob_mean = torch.mean(prob, dim = 0)
    #print(torch.argmax(prob_mean, 0))

    label=idx_label_dict[torch.argmax(prob_mean, 0).detach().cpu().item()]
    p_t.append(poem_df['text'][index])
    l_t.append(label)

100%|██████████| 1149/1149 [00:20<00:00, 55.78it/s]


In [10]:
poem_df['emotion_split_sen'] = l

In [11]:
poem_df['emotion_total_sen'] = l_t

In [12]:
poem_df['emotion_split_sen'].value_counts()

슬픔    406
기쁨    384
분노    176
불안    100
당황     60
상처     23
Name: emotion_split_sen, dtype: int64

In [13]:
poem_df['emotion_total_sen'].value_counts()

슬픔    592
기쁨    277
불안     84
분노     82
당황     79
상처     35
Name: emotion_total_sen, dtype: int64

In [63]:
poem_df = poem_df.drop(['emotion'],axis=1)

In [65]:
poem_df.to_csv('sinchoon+nonpub+400poems+emotion.csv',index=False)